In [41]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


Function #1

In [43]:
def pluralize_gift(gift):
    """
    Returns plural of a noun
    
    Parameters
    ----------
    gift: str
        A noun
        
    Return
    ------
    str
        Plural version
    """
    
    # Special cases
    if gift == "goose":
        gift = "geese"
    elif gift == "maid": 
        gift = "maids"
    # Words ending in 'y' preceded by a consonant, replace 'y' with 'ies'
    elif gift.endswith("y") and gift[-2] not in "aeiou":
        gift = gift[:-1] + "ies"
    # Regular case
    else:
        gift = gift + "s"

    return gift

# Test the function with examples
print(pluralize_gift("goose"))  # Should return 'geese'
print(pluralize_gift("baby"))   # Should return 'babies'
print(pluralize_gift("tree"))   # Should return 'trees'

# For vectorized operation
xmas['Gift.Item'].apply(pluralize_gift)

geese
babies
trees


0     partridges
1          doves
2           hens
3          birds
4          rings
5          geese
6          swans
7          maids
8         ladies
9          lords
10        pipers
11      drummers
Name: Gift.Item, dtype: object

Function #2 

In [44]:
# Dictionary 

# Dictionary mapping from ordinal to cardinal numbers
ordinal_to_cardinal = {
    'first': 'one',
    'second': 'two',
    'third': 'three',
    'fourth': 'four',
    'fifth': 'five',
    'sixth': 'six',
    'seventh': 'seven',
    'eighth': 'eight',
    'ninth': 'nine',
    'tenth': 'ten',
    'eleventh': 'eleven',
    'twelfth': 'twelve'
}

In [45]:
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Constructs and returns a phrase based on the provided elements.
    
    Parameters
    ----------
    num : int
        The day number.
    num_word : str
        The day number in words.
    item : str
        The gift item.
    verb : str
        The action associated with the gift.
    adjective : str
        Descriptive word for the gift.
    location : str
        The location associated with the gift.
    
    Return
    ------
    str
        The complete phrase.
    """

    # Step 1: Replace NAs
    verb = "" if pd.isna(verb) else verb
    adjective = "" if pd.isna(adjective) else adjective
    location = "" if pd.isna(location) else location
  
    # Step 2: Pluralize the gift
    if num > 1:
        item = pluralize_gift(item)
  
    # Step 3: Starts with a vowel?
    starts_with_vowel = item[0].lower() in 'aeiou'
    
    num_word_cardinal = ordinal_to_cardinal.get(num_word, num_word)

    # Step 4: Adjusting the dat, the first day? 
    # whether the item starts with a vowel.
    if num == 1:
        day_prefix = "an" if starts_with_vowel else "a"
    else:
        day_prefix = num_word_cardinal
  
    # Step 5: Making the phrase
    pieces = [day_prefix, item, verb, adjective, location]
    # Filter out
    phrase = " ".join(filter(None, pieces))

    return phrase.strip()  # Remove any extra spaces

# Test the function with a given example
test_phrase = make_phrase(
    num=10, 
    num_word="ten", 
    item="lord", 
    verb="a-leaping", 
    adjective="", 
    location=""
)
print(test_phrase)  # Should print "ten lords a-leaping"




ten lords a-leaping


In [46]:
twelvee =make_phrase(12, 'twelfth', 'drummer', 'drumming', '', '')
twelvee

'twelve drummers drumming'

In [47]:
xmas['Full.Phrase'] = xmas.apply(lambda row: make_phrase(row['Day'], row['Day.in.Words'], row['Gift.Item'], row['Verb'], row['Adjective'], row['Location']), axis=1)
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Full.Phrase
0,1,first,partridge,NaN,NaN,in a pear tree,a partridge in a pear tree
1,2,second,dove,NaN,turtle,NaN,two doves turtle
2,3,third,hen,NaN,french,NaN,three hens french
3,4,fourth,bird,NaN,calling,NaN,four birds calling
4,5,fifth,ring,NaN,golden,NaN,five rings golden
5,6,sixth,goose,a-laying,NaN,NaN,six geese a-laying
6,7,seventh,swan,a-swimming,NaN,NaN,seven swans a-swimming
7,8,eighth,maid,a-milking,NaN,NaN,eight maids a-milking
8,9,ninth,lady,dancing,NaN,NaN,nine ladies dancing
9,10,tenth,lord,a-leaping,NaN,NaN,ten lords a-leaping


Function # 3

In [48]:
def sing_day(dataset, num, phrase_col):
    """
    Constructs and returns the verse of the song for a specific day.
    
    Parameters
    ----------
    dataset : DataFrame
        The dataset containing the song's phrases.
    num : int
        The day number to sing about.
    phrase_col : str
        The name of the column containing the phrases for each day.
    
    Return
    ------
    str
        The complete verse for the specified day.
    """
  
    # Step 1: Setup the intro line
    num_to_words = {
        1: 'first', 2: 'second', 3: 'third', 4: 'fourth', 
        5: 'fifth', 6: 'sixth', 7: 'seventh', 8: 'eighth', 
        9: 'ninth', 10: 'tenth', 11: 'eleventh', 12: 'twelfth'
    }
    num_word = num_to_words[num]
    intro = "On the " + num_word + " day of Christmas, my true love sent to me:"
  
    # Step 2: Sing the gift phrases
    gifts = []
    for i in range(num, 0, -1):
        phrase = dataset.loc[i-1, phrase_col]  # assuming the dataset's first row is 0
        if i == 1 and num > 1:  # On days other than the first, we add 'and' before the last gift.
            phrase = "and " + phrase
        gifts.append(phrase)
    
    # Add commas where necessary
    gifts_with_commas = [gift + ',' for gift in gifts[:-1]] + [gifts[-1]]  # The last gift doesn't get a comma.
    
    # Step 3: Put it all together and return
    full_lyric = intro + '\n' + '\n'.join(gifts_with_commas) + '.'  # Add a period after the final gift.

    return full_lyric

# Test the function with the third day as an example
print(sing_day(xmas, 3, "Full.Phrase"))


On the third day of Christmas, my true love sent to me:
three hens french,
two doves turtle,
and a partridge in a pear tree.


In [52]:
# Full 12 Days of Christmas song. 
for day in range(1, 13):  # For each of the 12 days of Christmas
    verse = sing_day(xmas, day, "Full.Phrase")  # Get the verse
    print(verse)  
    print()

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two doves turtle,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three hens french,
two doves turtle,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
four birds calling,
three hens french,
two doves turtle,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
five rings golden,
four birds calling,
three hens french,
two doves turtle,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five rings golden,
four birds calling,
three hens french,
two doves turtle,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five rings golden,
four birds calling,
three hens french,
two doves turtle,
and a 

Testing the functions with the second dataset.

In [36]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN
5,6,sixth,grader,grading,NaN,NaN
6,7,seventh,senior,stressing,NaN,NaN
7,8,eighth,mom,a-calling,NaN,NaN
8,9,ninth,party,bumping,NaN,NaN
9,10,tenth,load,of laundry,NaN,NaN


In [55]:
xmas2['Full.Phrase'] = xmas2.apply(lambda row: make_phrase(row['Day'], row['Day.in.Words'], row['Gift.Item'], row['Verb'], row['Adjective'], row['Location']), axis=1)
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Full.Phrase
0,1,first,email,NaN,NaN,from Cal Poly,an email from Cal Poly
1,2,second,point,NaN,meal,NaN,two points meal
2,3,third,pen,NaN,lost,NaN,three pens lost
3,4,fourth,review,NaN,course,NaN,four reviews course
4,5,fifth,exam,NaN,practice,NaN,five exams practice
5,6,sixth,grader,grading,NaN,NaN,six graders grading
6,7,seventh,senior,stressing,NaN,NaN,seven seniors stressing
7,8,eighth,mom,a-calling,NaN,NaN,eight moms a-calling
8,9,ninth,party,bumping,NaN,NaN,nine parties bumping
9,10,tenth,load,of laundry,NaN,NaN,ten loads of laundry


In [56]:
for day in range(1, 13):  # For each of the 12 days of Christmas
    verse = sing_day(xmas2, day, "Full.Phrase")  # Get the verse
    print(verse)  
    print()

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two points meal,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three pens lost,
two points meal,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four reviews course,
three pens lost,
two points meal,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five exams practice,
four reviews course,
three pens lost,
two points meal,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five exams practice,
four reviews course,
three pens lost,
two points meal,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five exams practice,
four reviews course,
three pens lost,
two points meal,
and an email from Cal Poly.

On t